In [7]:
import casadi as ca

# Define the initial system matrices (you can modify this as needed)
A = ca.MX(ca.DM([[1.1, 0], [0, 0.95]]))
B = ca.MX(ca.DM([[0], [1]]))
Q = ca.MX(ca.DM([[1, 0], [0, 1]]))
R = ca.MX(ca.DM(1))
P = ca.MX(ca.DM([[1, 0], [0, 1]]))

N = 10  # Prediction horizon
n_states = A.shape[0]
n_controls = B.shape[1]

# Set up symbolic variables
x_0 = ca.MX.sym('x_0', n_states)
U = ca.MX.sym('U', n_controls, N)  # Stack of control actions over the horizon

# Discretize the system dynamics over the prediction horizon
X = [x_0]
for i in range(N):
    X.append(A@X[-1] + B@U[:, i])

# Formulate the cost function
cost = X[-1].T @ P @ X[-1]
for i in range(N):
    cost += X[i].T @ Q @ X[i] + U[:, i].T @ R @ U[:, i]

# Define the optimization problem
nlp = {'x': U, 'f': cost, 'g': [], 'p': x_0}
opts = {'ipopt': {'print_level': 5}, 'print_time': 0}
solver = ca.nlpsol('solver', 'ipopt', nlp, opts)

# Solve the optimization problem
x0_val = ca.MX(ca.DM([[3], [1]]))
U0 = ca.DM.zeros(n_controls, N)  # initial guess for control actions
res = solver(x0=U0, lbx=[], ubx=[], lbg=[], ubg=[], p=x0_val)
#optimal_controls = optimal_controls = ca.DM(res['x']).full()
print(type(res['x']))
print(res['f'])  # Objective function value
print(res['g'])  # Constraints (should be empty in your case)
print(solver.stats())  # Solver statistics


# ... use optimal_controls in your system ...


<class 'casadi.casadi.MX'>
solver(zeros(1x10), [3, 1], zeros(1x10), zeros(1x10), 0x1, 0x1, zeros(1x10), 0x1){1}
0x1
{'iter_count': 544106784, 'n_call_callback_fun': 0, 'n_call_nlp_f': 0, 'n_call_nlp_g': 0, 'n_call_nlp_grad': 0, 'n_call_nlp_grad_f': 0, 'n_call_nlp_hess_l': 0, 'n_call_nlp_jac_g': 0, 'return_status': 'Unset', 'success': False, 't_proc_callback_fun': 0.0, 't_proc_nlp_f': 0.0, 't_proc_nlp_g': 0.0, 't_proc_nlp_grad': 0.0, 't_proc_nlp_grad_f': 0.0, 't_proc_nlp_hess_l': 0.0, 't_proc_nlp_jac_g': 0.0, 't_wall_callback_fun': 0.0, 't_wall_nlp_f': 0.0, 't_wall_nlp_g': 0.0, 't_wall_nlp_grad': 0.0, 't_wall_nlp_grad_f': 0.0, 't_wall_nlp_hess_l': 0.0, 't_wall_nlp_jac_g': 0.0, 'unified_return_status': 'SOLVER_RET_UNKNOWN'}
